In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
import timeit

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

def cpu():
  with tf.device('/cpu:0'):
    random_image_cpu = tf.random.normal((100, 100, 100, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    return tf.math.reduce_sum(net_cpu)

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)
  
# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
cpu()
gpu()

<tf.Tensor: shape=(), dtype=float32, numpy=1418.4008>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.io import loadmat
from tensorflow import keras
from tensorflow.keras import layers
import gc
tf.keras.backend.set_floatx('float64')

def stagger_data(data, h):
    """
    >>> i = np.array([[1, 2, 3, 4, 5, 6], [7, 8, 9, 10, 11, 12]])
    >>> stagger_data(i, [1, 3])
    (array([[ 3,  4,  5],
           [ 9, 10, 11],
           [ 1,  2,  3],
           [ 7,  8,  9]]), array([[ 4,  5,  6],
           [10, 11, 12]]))
    """
    h.sort()
    len_h = len(h)
    n, m = data.shape
    max_h = max(h)

    Y = data[:, max_h:]
    X = np.zeros((n * len_h, m - max_h), dtype=data.dtype)
    for i in range(len_h):
        X[i * n: i * n + n, :] = data[:, max_h - h[i]:m - h[i]]
    return X, Y


def remove_weekends(data, start=0, bs=36):
    _, m = data.shape
    n_day = int(m / bs)
    weekday = np.concatenate([np.arange(start, 7) % 7, np.arange(n_day) % 7])[:n_day]
    weekday = np.repeat(weekday, bs)
    return data[:, weekday < 5]


def get_flow1(od, s, dir='o'):
    """Get the flow of station `s`"""
    n = od.shape[0]
    if dir == 'o':
        idx = np.arange(s, n, 159)
    elif dir == 'd':
        idx = np.arange((s * 159), (s * 159 + 159))
    return np.sum(od[idx, :], axis=0)


def od2flow(od, s_list=None, dir='o'):
    if s_list is None:
        s_list = range(159)

    n_s = len(s_list)
    flow = np.zeros((n_s, od.shape[1]), dtype=np.float32)
    for i, s in enumerate(s_list):
        flow[i, :] = get_flow1(od, s, dir)
    return flow


def RMSE(f0, f1, axis=None):
    return np.sqrt(np.mean((f0 - f1) ** 2, axis))
  

def start_end_idx(start, end, weekend=False, night=False):
    date = pd.period_range('2017-07-01', '2017-09-30 23:30', freq='30T')
    date = date.to_timestamp()
    if not night:
        date = date[date.hour >= 6]
    if not weekend:
        date = date[date.weekday < 5]
    idx = pd.DataFrame(data=np.arange(date.shape[0]), index=date)
    return idx.loc[start:end, :].values.ravel()

# Load data

In [ ]:
data0 = loadmat('drive//MyDrive//data//OD_3m.mat')
data0 = data0['OD']
data0 = remove_weekends(data0, start=5)

# Subtract the mean of the training set
data = data0.astype(np.float64)
data_mean = data[:, 0:20*36].reshape([159*159, 36, -1], order='F')
data_mean = data_mean.mean(axis=2)
for i in range(65):
    data[:,i*36:(i+1)*36] = data[:,i*36:(i+1)*36] - data_mean

train_idx = start_end_idx('2017-07-03', '2017-07-28', weekend=False, night=False)
validate_idx = start_end_idx('2017-07-31', '2017-08-11', weekend=False, night=False)
test_idx = start_end_idx('2017-08-14', '2017-08-25', weekend=False, night=False)

flow0 = od2flow(data)
flow = np.zeros((flow0.shape[0]*2, flow0.shape[1]), dtype=flow0.dtype)
flow[0:flow0.shape[0], :] = flow0
flow[flow0.shape[0]:, 1:] = flow0[:, 0:-1]

h = [3, 4, 5, 6, 7, 8, 9, 10]
X_train, Y_train = stagger_data(data[:, train_idx], h)
m_train = X_train.shape[1]
X_train = np.concatenate([X_train, flow[:, train_idx][:, -m_train-1:-1]/159]).T
Y_train = Y_train.T

X_validate, Y_validate = stagger_data(data[:, np.concatenate([train_idx[-max(h):], validate_idx])], h)
X_validate = np.concatenate([X_validate, flow[:, validate_idx-1]/159]).T
Y_validate = Y_validate.T

X_test, Y_test = stagger_data(data[:, np.concatenate([validate_idx[-max(h):], test_idx])], h)
X_test = np.concatenate([X_test, flow[:, test_idx-1]/159]).T
Y_test = Y_test.T

# Select model order

In [ ]:
def create_net(n_input=159*159*10, n_hidden=50, activation=None):
  seq = keras.Sequential(
      [
      layers.Dense(n_hidden, input_shape=(n_input,), 
                   activation=activation,
                  #  kernel_regularizer=tf.keras.regularizers.L2(0.0001)
                   ),
      layers.Dense(159*159,
                  #  kernel_regularizer=tf.keras.regularizers.L2(0.0001)
                   )
      ]
  )
  return seq

call_back = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0, patience=20, verbose=0, mode='auto',
    baseline=None, restore_best_weights=True
)

In [ ]:
activation_list = ['linear', 'sigmoid', 'relu']
n_hidden_list = np.linspace(10, 100, 10, dtype=np.int)
best_e = 10000

for activation in activation_list:
    for n_hidden in n_hidden_list:
        tf.keras.backend.clear_session()
        tf.random.set_seed(1)
        seq = create_net(n_input=X_train.shape[1], n_hidden=n_hidden, activation=activation)
        gc.collect()
        seq.compile(loss="mean_squared_error", optimizer="RMSprop")
        seq.fit(
            x=X_train,
            y=Y_train,
            batch_size=32,
            epochs=200,
            verbose=0,
            shuffle=True,
            validation_data=(X_validate, Y_validate),
            callbacks=[call_back],
        )
        predict_validate = seq.predict(X_validate)
        rmse = RMSE(predict_validate, Y_validate)

        if rmse<best_e:
            best_e = rmse
            best_model = seq
        print('Activation {} hidden_layers {}, RMSE {}, current best RMSE {}'.format(activation, n_hidden, rmse, best_e))

Activation linear hidden_layers 10, RMSE 3.0939109330214096, current best RMSE 3.0939109330214096
Activation linear hidden_layers 20, RMSE 3.095054706964265, current best RMSE 3.0939109330214096
Activation linear hidden_layers 30, RMSE 3.0936376910944134, current best RMSE 3.0936376910944134
Activation linear hidden_layers 40, RMSE 3.0977290257856964, current best RMSE 3.0936376910944134
Activation linear hidden_layers 50, RMSE 3.090493691030671, current best RMSE 3.090493691030671
Activation linear hidden_layers 60, RMSE 3.0957928095770684, current best RMSE 3.090493691030671
Activation linear hidden_layers 70, RMSE 3.092542771713564, current best RMSE 3.090493691030671
Activation linear hidden_layers 80, RMSE 3.093912434060778, current best RMSE 3.090493691030671
Activation linear hidden_layers 90, RMSE 3.0914988669704657, current best RMSE 3.090493691030671
Activation linear hidden_layers 100, RMSE 3.1083641925081302, current best RMSE 3.090493691030671
Activation sigmoid hidden_lay

In [ ]:
# Retrain the model by train + validate set, using the best hyperparameters.
X_train_new, Y_train_new = stagger_data(data[:, :test_idx[1]], h)
m_train_new = X_train_new.shape[1]
X_train_new = np.concatenate([X_train_new, flow[:, :test_idx[1]][:, -m_train_new-1:-1]/159]).T
Y_train_new = Y_train_new.T

# Train the model
tf.keras.backend.clear_session()
tf.random.set_seed(1)
seq = create_net(n_input=X_train_new.shape[1], n_hidden=50, activation='linear')
gc.collect()
seq.compile(loss="mean_squared_error", optimizer="RMSprop")
seq.fit(
    x=X_train_new,
    y=Y_train_new,
    batch_size=32,
    epochs=200,
    verbose=0,
    shuffle=True,
    validation_split=0.2,
    callbacks=[call_back],
)

tf.keras.backend.clear_session()
gc.collect()

seq.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 50)                10128350  
_________________________________________________________________
dense_1 (Dense)              (None, 25281)             1289331   
Total params: 11,417,681
Trainable params: 11,417,681
Non-trainable params: 0
_________________________________________________________________


# Multistep forecast for the test set

In [ ]:
# One-step forecast
result1 = seq.predict(np.concatenate([X_validate, X_test], axis=0))
tf.keras.backend.clear_session()
gc.collect()

print('RMSE on test data: {}'.format(RMSE(np.concatenate([Y_validate, Y_test], axis=0)[360:, :], result1[360:, :])))

RMSE on test data: 3.2557383521303676


In [ ]:
# Two-step forecast
tf.keras.backend.clear_session()
gc.collect()
n = data.shape[0]
nh = len(h)
X2 = np.concatenate([X_validate, X_test])

# Reuse one-step forecast OD
X2[3:, 0:n] = result1[0:-3, :]

# Reuse one-step forecast flow
X2[3:, n*nh:n*nh+159] = od2flow(result1[2:-1, :].T).T/159

# Forecast
result2 = seq.predict(X2)
tf.keras.backend.clear_session()
gc.collect()

print('RMSE on test data: {}'.format(RMSE(np.concatenate([Y_validate, Y_test], axis=0)[360:, :], result2[360:, :])))

RMSE on test data: 3.274448125602289


In [ ]:
# Three-step forecast
n = data.shape[0]
nh = len(h)
X3 = np.concatenate([X_validate, X_test])
# Reuse one-step forecast OD
X3[4:, 0:n] = result1[1:-3, :]
X3[4:, n:2*n] = result1[0:-4, :]

# Reuse one-step forecast flow
X3[4:, n*nh:n*nh+159] = od2flow(result1[3:-1, :].T).T/159
X3[4:, n*nh+159:] = od2flow(result1[2:-2, :].T).T/159

# Forecast
result3 = seq.predict(X3)
tf.keras.backend.clear_session()
gc.collect()

print('RMSE on test data: {}'.format(RMSE(np.concatenate([Y_validate, Y_test], axis=0)[360:, :], result3[360:, :])))

RMSE on test data: 3.2855431714645014


# Add mean back and save results to file

In [ ]:
result1=result1[360:, :].T
for i in range(result1.shape[1]):
  result1[:,i] += data_mean[:, i%36]

result2=result2[360:, :].T
for i in range(result2.shape[1]):
  result2[:,i] += data_mean[:, i%36]

result3=result3[360:, :].T
for i in range(result3.shape[1]):
  result3[:,i] += data_mean[:, i%36]

np.savez_compressed('/content/drive/MyDrive/data/OD_FNN_step1.npz', data=result1)
np.savez_compressed('/content/drive/MyDrive/data/OD_FNN_step2.npz', data=result2)
np.savez_compressed('/content/drive/MyDrive/data/OD_FNN_step3.npz', data=result3)